<a href="https://colab.research.google.com/github/kjapan87/TextGenerationV1/blob/main/Text_Generation_Try_2020_10_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#STEP-1:::Create a function to Read training text file
def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
    return str_text

In [ ]:

#STEP-2::: Reading the actual text
training_text_data = read_file('moby_dick_four_chapters.txt')

In [ ]:
                 ###TOKENIZE & CLEAN TEXT###
import spacy
nlp = spacy.blank("en")


def Pre_Process(doc_text): #defining a function to pre-procecss training_text_data, by removing the punctuations and converting upper case to lower case
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']


spacy_tokens = Pre_Process(training_text_data)
len(spacy_tokens)


11338

In [ ]:
train_len = 25+1 #defining and organizing the tokens into sliding window sequence,25 training words then 1 predicted word

text_sequence = [] #declare an empty list
for i in range(train_len,len(spacy_tokens)):
    seq = spacy_tokens[i-train_len:i]  # Defining sliding windows of 25 words
  
    text_sequence.append(seq)          # Actually creating sliding windows of 25 words
    
' '.join(text_sequence[0])#no use, just for checking for-loop is working or not
' '.join(text_sequence[1])#no use, just for checking for-loop is working or not
' '.join(text_sequence[2])#no use, just for checking for-loop is working or not
' '.join(text_sequence[3])#no use, just for checking for-loop is working or not

len(text_sequence)

11312

In [ ]:
                   ###KERAS TOKENIZATION###
from keras.preprocessing.text import text_to_word_sequence,Tokenizer
obj4 = Tokenizer()
obj4.fit_on_texts(text_sequence)
keras_tokens = obj4.texts_to_sequences(text_sequence) # converts a text file, into tokens & also removes basic punctuations tabs etc.(same as ln23 function)
keras_tokens[0]


for i in keras_tokens[0]:
    print(f'{i} : {obj4.index_word[i]}')
    
vocab_size = len(obj4.word_counts)

956 : call
14 : me
263 : ishmael
51 : some
261 : years
408 : ago
87 : never
219 : mind
129 : how
111 : long
954 : precisely
260 : having
50 : little
43 : or
38 : no
315 : money
7 : in
23 : my
546 : purse
3 : and
150 : nothing
259 : particular
6 : to
2712 : interest
14 : me
24 : on


In [ ]:
vocab_size

2717

In [ ]:
       ###CONVERT KERAS_TOKENS LIST TO NUMPY MATRIX###
import numpy as np
keras_tokens = np.array(keras_tokens) # convert keras_token(list) to keras_token(numpy arrays)
#np.array or .str converts to any data type

            ###ENCODE KERAS_TOKEN NUMPY WORD MATRIX AS INTEGER MATRIX###
#from keras.preprocessing.text import one_hot
#vocab_size = len(keras_tokens.word_counts)
#encoded_keras_tokens = one_hot(training_text_data, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,split=' ')       


In [ ]:
       ###CONVERT KERAS_TOKEN NUMPY INTEGER MATRIX TO BINARY MATRIX###
from numpy import array
from numpy import argmax
from keras.utils import to_categorical
#encoded_keras_tokens_binary = to_categorical(encoded_keras_tokens)
#inverted_encoded_keras_tokens_binary = argmax(encoded_keras_tokens_binary[0]) #invert encoding

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

def create_model(vocab_size, sequence_length):
    model = Sequential()
    model.add(Embedding(vocab_size,sequence_length, input_length=sequence_length))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(64))
    model.add(Dense(64, activation='relu'))

    model.add(Dense(vocab_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model

In [ ]:
from keras.utils import to_categorical
                
x = keras_tokens[0:11313,0:25]
y = keras_tokens[:,-1]
y = to_categorical(y)

sequence_length = x.shape[1]

In [ ]:
sequence_length

25

In [ ]:
vocab_size

2717

In [ ]:
model = create_model(vocab_size, sequence_length)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 25, 25)            67925     
_________________________________________________________________
lstm_30 (LSTM)               (None, 25, 64)            23040     
_________________________________________________________________
lstm_31 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_7 (Dense)              (None, 2717)              176605    
Total params: 304,754
Trainable params: 304,754
Non-trainable params: 0
_________________________________________________________________


In [ ]:
vocab_size

2717